In [141]:
# For using custom module.
import sys
sys.path.append('../custom_library')

In [142]:
import pymysql.cursors
import pandas as pd
import numpy as np
import connect_to_db as cn
import json
from itertools import combinations
from sklearn.feature_extraction.text import TfidfVectorizer
import re
import csv
import parmap

In [62]:
def check_body(document):
    p = re.compile('[a-zA-Z]')
    for text in document[:]:
        # 특수문자, 이모지 등으로만 이루어진 text를 거르기 위한 code.
        if p.search(text) is None:
            document.remove(text)
        # 특수문자와 알파벳 하나(ex. :p)로만 이루어진 text를 거르기 위한 code.
        elif len(re.sub('[^a-zA-Z]',' ',text).strip()) == 1:
            document.remove(text)
        else:
            text_list = text.split('.')
            length = len(text_list)
            for i in range(length):
                if len(text_list[i]) > 1:
                    break
                if length - 1 == i:
                    document.remove(text)
    
    return document

In [86]:
def tf_idf_similarity(sql, column_name):
    result_df = cn.select_query_result_to_df(sql)
    document = list(np.array(result_df[column_name].tolist()))
    document = check_body(document)

    # comment/body list가 빈 list인지를 체크해서 빈 list이면 -1 return.
    if not document:
        return -1
    
    # check_body() 함수를 거친 후, 이모지로만 이루어진 comment/post title이 지워져서
    # 하나의 text만 남았을 경우. 구분을 위해 -2 return.
    if len(document) == 1:
        return -2

    tfidf_vectorizer = TfidfVectorizer(min_df=1)
    tfidf_matrix = tfidf_vectorizer.fit_transform(document)

    document_distances = (tfidf_matrix * tfidf_matrix.T)
    
    result_array = document_distances.toarray()
    
    similarity = 0
    count = 0
    
    for i in range(len(result_array)):
        for j in range(len(result_array[i])):
            if i < j:
                similarity += result_array[i][j]
                count += 1
    
    return (similarity / count)

In [126]:
def tf_idf_similarity_from_list(document):
    document = check_body(document)

    # comment/body list가 빈 list인지를 체크해서 빈 list이면 -1 return.
    if not document:
        return -1
    
    # check_body() 함수를 거친 후, 이모지로만 이루어진 comment/post title이 지워져서
    # 하나의 text만 남았을 경우. 구분을 위해 -2 return.
    if len(document) == 1:
        return -2

    tfidf_vectorizer = TfidfVectorizer(min_df=1)
    tfidf_matrix = tfidf_vectorizer.fit_transform(document)

    document_distances = (tfidf_matrix * tfidf_matrix.T)
    
    result_array = document_distances.toarray()
    
    similarity = 0
    count = 0
    
    for i in range(len(result_array)):
        for j in range(len(result_array[i])):
            if i < j:
                similarity += result_array[i][j]
                count += 1
    
    return (similarity / count)

In [30]:
def similarity_main(index):
    # similarity column들은 모두 null 값이라 가정하고 실행.
    # sql = f'select author from mentor where comment_cnt > 1 and is_valid = 1;'
    sql = f'select author from mentor where comment_cnt > 1;'
    result_df = cn.select_query_result_to_df(sql)
    authors = np.array(result_df['author'].astype(str).values.tolist())

    result_for_csv = []
    
    if index % 100000 == 0:
        start_index = index - 100000
    else:
        start_index = index - (index % 100000)
        
    for i in range(start_index, index):
        sql1 = f"select body from comments c, posts p where p.post_key = c.link_key and c.author = '{authors[i]}' and link_key = parent_key and p.is_valid_author=1;"
        comments_similarity = tf_idf_similarity(sql1, 'body')
        
        sql2 = f"select distinct p.post_key, p.title from posts p, comments c where p.post_key = c.link_key and c.author = '{authors[i]}' and c.link_key = c.parent_key and p.is_valid_author=1;";
        posts_similarity = tf_idf_similarity(sql2, 'title')        
        
        result_for_csv.append([authors[i], comments_similarity, posts_similarity])

    fields = ['author', 'comments_similarity', 'posts_similarity']
    cn.write_csv_for_db_update(f'similarity_final_{index}.csv', fields, result_for_csv)

In [66]:
def mentee_posts_similarity_main(index):
    # similarity column들은 모두 null 값이라 가정하고 실행.
    # sql = f'select author from mentor where comment_cnt > 1 and is_valid = 1;'
    sql = f'select author from mentee where post_cnt > 1;'
    result_df = cn.select_query_result_to_df(sql)
    authors = np.array(result_df['author'].astype(str).values.tolist())

    result_for_csv = []
    
    if index % 100000 == 0:
        start_index = index - 100000
    else:
        start_index = index - (index % 100000)
        
    for i in range(start_index, index):
        sql = f"select title from posts where author='{authors[i]}';"
        posts_similarity = tf_idf_similarity(sql, 'title')
        
        result_for_csv.append([authors[i], posts_similarity])

    fields = ['author', 'posts_similarity']
    cn.write_csv_for_db_update(f'posts_similarity_mentee_{index}.csv', fields, result_for_csv)

In [87]:
def mentee_comments_similarity_main(index):
    # similarity column들은 모두 null 값이라 가정하고 실행.
    # sql = f'select author from mentor where comment_cnt > 1 and is_valid = 1;'
    sql = f'select author from mentee where comment_cnt > 1;'
    result_df = cn.select_query_result_to_df(sql)
    authors = np.array(result_df['author'].astype(str).values.tolist())

    result_for_csv = []
    
    if index % 100000 == 0:
        start_index = index - 100000
    else:
        start_index = index - (index % 100000)
        
    for i in range(start_index, index):        
        sql = f"select body from posts p, comments c where p.post_key = c.link_key and c.is_valid=1 and c.is_valid_author=1 and p.author='{authors[i]}' and c.link_key = c.parent_key and p.author <> c.author;";
        comments_similarity = tf_idf_similarity(sql, 'body')        
        
        result_for_csv.append([authors[i], comments_similarity])

    fields = ['author', 'comments_similarity']
    cn.write_csv_for_db_update(f'comments_similarity_mentee_{index}.csv', fields, result_for_csv)

In [16]:
def posts_similarity_main(index):
    sql = f'select post_key from about_posts where comment_cnt > 1;'
    result_df = cn.select_query_result_to_df(sql)
    post_keys = np.array(result_df['post_key'].astype(str).values.tolist())
    
    result_for_csv = []
    
    if index % 100000 == 0:
        start_index = index - 100000
    else:
        start_index = index - (index % 100000)
        
    for i in range(start_index, index):
        sql = f"select body from comments where link_key = '{post_keys[i]}' and link_key = parent_key and is_valid=1 and is_valid_author=1;"
        comments_similarity = tf_idf_similarity(sql, 'body')
        
        result_for_csv.append([post_keys[i], comments_similarity])
    
    fields = ['post_key', 'comments_similarity']
    cn.write_csv_for_db_update(f"about_posts_comments_similarity_{index}.csv", fields, result_for_csv)

In [88]:
# index_list = [100000, 200000, 300000, 349103]
# index_list = [100000, 200000, 227032]
# index_list = [100000, 200000, 300000, 359737]
# index_list = [100000, 200000, 224406]
# index_list = [100000, 190746]
index_list = [100000, 200000, 228602]

if __name__ == '__main__':
    # multi processing.
    parmap.map(mentee_comments_similarity_main, index_list, pm_pbar=True, pm_processes=3)
    # parmap.map(posts_similarity_main, index_list, pm_pbar=True, pm_processes=4)

100%|██████████| 3/3 [06:00<00:00, 120.08s/it]


In [138]:
def random_similarity_main(cnt):
    # order by rand()를 사용할 경우, 너무 많은 시간 소요.
    # 일단 query로 모든 comments, posts를 가져온 후에 random으로 choose.
    result_for_csv = []
    
    sql1 = f"select body from comments where is_valid=1 and is_valid_author=1;"
    comments_df = cn.select_query_result_to_df(sql1)
    
    sql2 = f"select p.title from posts p, comments c where p.post_key = c.link_key and p.is_valid=1 and p.is_valid_author=1;"
    posts_df = cn.select_query_result_to_df(sql2)
    
    for i in range(1000):
        random_comments_similarity = -1
        random_posts_similarity = -1
        
        while random_comments_similarity < 0:
            comments = list(np.array(comments_df.sample(cnt)['body'].tolist()))
            random_comments_similarity = tf_idf_similarity_from_list(comments)
        
        while random_posts_similarity < 0:
            posts = list(np.array(posts_df.sample(cnt)['title'].tolist()))
            
            random_posts_similarity = tf_idf_similarity_from_list(posts)
        
        result_for_csv.append([random_comments_similarity, random_posts_similarity])
    
    fields = ['random_comments_similarity', 'random_posts_similarity']
    cn.write_csv_for_db_update(f'random_similarity_{cnt}.csv', fields, result_for_csv)

In [140]:
random_similarity_main(4)